# 비디오 분석 작업 호출 - 프레임 샘플 기반

도구의 UI를 사용하거나 기본 API를 직접 호출하여 비디오 분석 작업을 생성할 수 있습니다. 이 노트북은 필요한 메타데이터를 준비하고 Lambda 함수를 통해 작업을 호출하는 방법의 예제를 제공합니다.

In [1]:
import boto3
import json
import uuid

In [2]:
lambda_client = boto3.client("lambda")
s3 = boto3.client("s3")

In [3]:
session = boto3.session.Session()
sts = session.client("sts")
# 현재 계정 ID 가져오기
account_id = sts.get_caller_identity()["Account"]
# 현재 리전 가져오기
region = session.region_name

In [4]:
S3_BUCKET = f"bedrock-mm-{account_id}-{region}"
S3_KEY_TASK_TEMPLATE = "tasks/{task_id}/upload/{file_name}"

LAMBDA_FUN_NAME_FRAME_ANALYSIS_START_TASK = "bedrock-mm-extr-srv-api-start-task"
LAMBDA_FUN_NAME_FRAME_ANALYSIS_DELETE_TASK = "bedrock-mm-extr-srv-api-delete-task"

## 프레임 기반 비디오 작업 호출

프레임 샘플 기반 비디오 분석 워크플로우에는 다음 매개변수가 필요합니다:
- local_path: 분석할 비디오가 저장된 로컬 디스크의 위치
- task_id: 비디오 작업의 고유 식별자
- s3_key: 비디오가 저장될 S3 객체 키
- task_name: 도구 UI에 표시될 작업 이름 (제공되지 않으면 비디오 파일 이름이 기본값)
- requested_by: 추적 목적으로 요청을 제출한 사용자

In [5]:
local_path = "./samples/content-moderation-demo.mp4"

# task_id 생성
task_id = str(uuid.uuid4())

# 대상 s3 키 구성
file_name = local_path.split('/')[-1]
s3_key = S3_KEY_TASK_TEMPLATE.format(task_id=task_id, file_name=file_name)

# 요청에 필요한 기타 값
task_name = file_name # 작업 이름 설정, 기본값은 파일 이름 사용
request_by = "workshop-user" # 요청 사용자 이름 설정

로컬 디스크에서 S3로 비디오 업로드

In [6]:
s3.upload_file(local_path, S3_BUCKET, s3_key)

다음 JSON 샘플은 프레임 샘플 기반 작업에 필요한 세부 매개변수를 보여주는 템플릿 역할을 합니다. 도구의 UI를 통해서도 이를 관리할 수 있습니다.

In [7]:
request = {
  "TaskId": task_id,
  "FileName": file_name,
  "TaskName": task_name,
  "Video": {
    "S3Object": {
      "Bucket": S3_BUCKET,
      "Key": s3_key
    }
  },
  "RequestBy": request_by,
  "PreProcessSetting": {
    "SampleMode": "even",
    "SampleIntervalS": 1,
    "SmartSample": True,
    "SimilarityMethod": "novamme",
    "SimilarityThreshold": 0.2
  },
  "ExtractionSetting": {
    "Audio": {
      "Transcription": True
    },
    "Vision": {
      "Frame": {
        "Enabled": True,
        "PromptConfigs": [
          {
            "name": "Content moderation",
            "toolConfig": {
              "tools": [
                {
                  "toolSpec": {
                    "name": "tool_result",
                    "description": "제공된 비디오에 안전하지 않은 콘텐츠가 포함되어 있는지 감지하고 다음 범주로 분류합니다.",
                    "inputSchema": {
                      "json": {
                        "type": "object",
                        "properties": {
                          "suggestive": {
                            "type": "boolean",
                            "description": "비디오에 상반신 노출 남성, 노출이 심한 의상 또는 기타 부적절한 장면이 포함되어 있는지 나타냅니다."
                          },
                          "tobacco": {
                            "type": "boolean",
                            "description": "비디오에 상반신 노출 남성, 노출이 심한 의상 또는 기타 성적으로 암시적인 콘텐츠를 포함하되 이에 국한되지 않는 장면이 포함되어 있는지 나타냅니다."
                          },
                          "alcohol": {
                            "type": "boolean",
                            "description": "비디오에 알코올과 관련된 장면이 포함되어 있는지 나타냅니다."
                          }
                        },
                        "required": [
                          "suggestive",
                          "tobacco",
                          "alcohol"
                        ]
                      }
                    }
                  }
                }
              ]
            },
            "inferConfig": {
              "maxTokens": 500,
              "temperature": 0.7,
              "topP": 0.1
            },
            "modelId": "amazon.nova-lite-v1:0",
            "prompt": "당신은 비디오를 검토하여 안전하지 않은 콘텐츠를 감지하는 콘텐츠 조정 전문가입니다."
          }
        ]
      }
    }
  }
}


Lambda 함수를 호출하여 작업 실행

In [8]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_FRAME_ANALYSIS_START_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps(request),
)

# Lambda 응답 파싱
response_payload = response["Payload"].read()
response_payload

b'{"statusCode": 200, "body": {"TaskId": "69dfd8a5-e550-4daf-8b58-0d3b23d24212"}}'

## 정리
delete-task 엔드포인트를 호출하여 업로드된 비디오(S3), S3에 생성된 모든 메타데이터 파일, DynamoDB에 저장된 메타데이터를 포함한 모든 작업 관련 상태를 제거합니다. MME 작업의 경우 S3 벡터 인덱스에서 비디오 관련 벡터도 삭제됩니다.

In [9]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_FRAME_ANALYSIS_DELETE_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps({"TaskId": task_id}),
)

# Lambda 응답 파싱
response_payload = response["Payload"].read()
response_payload

b'{"statusCode": 200, "body": "Deleting video task: 69dfd8a5-e550-4daf-8b58-0d3b23d24212"}'